### This the TFIDF version of the Hybrid Model

In [1]:
#make necessary imports
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [2]:
#Read the data
df=pd.read_csv('../../datasets/liar_tweaked/trainvectordata.csv')
testdf=pd.read_csv('../../datasets/liar_tweaked/testvectordata.csv')
validdf=pd.read_csv('../../datasets/liar_tweaked/validvectordata.csv')

In [3]:
x_train,y_train=df['statement'],df['label']
x_test,y_test=testdf['statement'],testdf['label']
x_valid,y_valid=validdf['statement'],validdf['label']

In [4]:
#get the tfidf
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(df['statement']) 
tfidf_test=tfidf_vectorizer.transform(testdf['statement'])  
tfidf_valid=tfidf_vectorizer.transform(validdf['statement'])  

In [5]:
#split for training and leave 40% for ANN training
classical_x_train,Left_for_ANN_x_train,classical_y_train,Left_for_ANN_y_train=train_test_split(tfidf_train, df['label'], test_size=0.4, random_state=7)

In [6]:
#prepare ANN train data, use 60% data to train classical ML models and get predictions on 
#remaining 40% data, use this remaining 40% data for ANN training
def prepareANNtrainData(x_train,y_train,x_test):
    clf = SVC()
    clf.fit(x_train, y_train)
    y_SVM = clf.predict(x_test)
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    y_RF = clf.predict(x_test)
    clf = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    clf.fit(x_train, y_train)
    y_KNN = clf.predict(x_test)
    clf = LogisticRegression(random_state = 0)
    clf.fit(x_train, y_train)
    y_LR = clf.predict(x_test)
    df=pd.DataFrame()
    df['SVM'],df['RF'],df['KNN'],df['LR']=y_SVM, y_RF, y_KNN, y_LR
    return df

In [7]:
#prepare ANN test data, get predictions of classical models on test and valid set and return these datasets to test
#ANN model
def prepareANNtestData(x_train,y_train,tfidf_test,tfidf_valid):
    clf = SVC()
    clf.fit(x_train, y_train)
    y_test_SVM = clf.predict(tfidf_test)
    y_valid_SVM = clf.predict(tfidf_valid)
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    y_test_RF = clf.predict(tfidf_test)
    y_valid_RF = clf.predict(tfidf_valid)
    clf = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    clf.fit(x_train, y_train)
    y_test_KNN = clf.predict(tfidf_test)
    y_valid_KNN = clf.predict(tfidf_valid)
    clf = LogisticRegression(random_state = 0)
    clf.fit(x_train, y_train)
    y_test_LR = clf.predict(tfidf_test)
    y_valid_LR = clf.predict(tfidf_valid)
    df_test=pd.DataFrame()
    df_valid=pd.DataFrame()
    df_test['SVM'],df_test['RF'],df_test['KNN'],df_test['LR']=y_test_SVM, y_test_RF, y_test_KNN, y_test_LR
    df_valid['SVM'],df_valid['RF'],df_valid['KNN'],df_valid['LR']=y_valid_SVM, y_valid_RF, y_valid_KNN, y_valid_LR
    return df_test,df_valid

In [8]:
#get ANN train data
ANNtrainData = prepareANNtrainData(classical_x_train,classical_y_train,Left_for_ANN_x_train)

In [9]:
ANNtrainData

,SVM,RF,KNN,LR
0,1,0,1,0
1,1,1,1,1
2,1,1,1,1
3,0,0,1,0
4,1,1,1,1
...,...,...,...,...
4087,0,1,0,0
4088,1,0,1,1
4089,1,1,1,1
4090,1,1,1,1


In [10]:
#build classifier
def build_classifier():
    clf=Sequential()
    clf.add(Dense(output_dim=2,init='uniform',activation='relu',input_dim=4))
    #clf.add(Dense(output_dim=2,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))
    clf.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return clf

In [11]:
#make necessary imports
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

#build ANN, use k fold cross validation
clf=KerasClassifier(build_fn=build_classifier, batch_size=10, nb_epoch=100)
accuracies=cross_val_score(estimator=clf, X=ANNtrainData,y=Left_for_ANN_y_train,cv=10,n_jobs=-1)

Using TensorFlow backend.


In [12]:
#see accuracies
accuracies

array([0.60243905, 0.52682924, 0.56723714, 0.54767728, 0.57946211,
       0.57457215, 0.58679706, 0.56479216, 0.58435208, 0.58679706])

In [13]:
#get ANN test and valid data
ANNtest,ANNvalid=prepareANNtestData(classical_x_train,classical_y_train,tfidf_test,tfidf_valid)

In [16]:
#fit on training data and check accuracies on both test and valid data
clf.fit(ANNtrainData,Left_for_ANN_y_train,batch_size=10, nb_epoch=100)
y_test_pred = clf.predict(ANNtest)
print('algorithm - test dataset accuracy - valid dataset accuracy')
print('Hybrid V3 - ' ,round(accuracy_score(y_test, y_test_pred),4), ' - ', end='')
y_test_pred = clf.predict(ANNvalid)
print(round(accuracy_score(y_valid, y_test_pred),4))

/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=4, units=2, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


Epoch 1/100
4092/4092 [==============================] - 3s 739us/step - loss: 0.6835 - accuracy: 0.5740
Epoch 2/100
4092/4092 [==============================] - 1s 327us/step - loss: 0.6748 - accuracy: 0.5721
Epoch 3/100
4092/4092 [==============================] - 1s 276us/step - loss: 0.6733 - accuracy: 0.5721
Epoch 4/100
4092/4092 [==============================] - 1s 309us/step - loss: 0.6721 - accuracy: 0.5775
Epoch 5/100
4092/4092 [==============================] - 1s 323us/step - loss: 0.6712 - accuracy: 0.5987
Epoch 6/100
4092/4092 [==============================] - 1s 363us/step - loss: 0.6705 - accuracy: 0.5987
Epoch 7/100
4092/4092 [==============================] - 1s 359us/step - loss: 0.6700 - accuracy: 0.5987
Epoch 8/100
4092/4092 [==============================] - 2s 394us/step - loss: 0.6696 - accuracy: 0.5997
Epoch 9/100
4092/4092 [==============================] - 1s 308us/step - loss: 0.6691 - accuracy: 0.6022
Epoch 10/100
4092/4092 [==============================]

4092/4092 [==============================] - 1s 341us/step - loss: 0.6677 - accuracy: 0.5997
Epoch 79/100
4092/4092 [==============================] - 1s 338us/step - loss: 0.6677 - accuracy: 0.5997
Epoch 80/100
4092/4092 [==============================] - 1s 331us/step - loss: 0.6677 - accuracy: 0.5997
Epoch 81/100
4092/4092 [==============================] - 1s 336us/step - loss: 0.6676 - accuracy: 0.5997
Epoch 82/100
4092/4092 [==============================] - 1s 331us/step - loss: 0.6677 - accuracy: 0.5997
Epoch 83/100
4092/4092 [==============================] - 1s 349us/step - loss: 0.6678 - accuracy: 0.5997
Epoch 84/100
4092/4092 [==============================] - 1s 353us/step - loss: 0.6678 - accuracy: 0.5997
Epoch 85/100
4092/4092 [==============================] - 1s 344us/step - loss: 0.6677 - accuracy: 0.5997
Epoch 86/100
4092/4092 [==============================] - 1s 352us/step - loss: 0.6679 - accuracy: 0.5997
Epoch 87/100
4092/4092 [==============================] - 1